# 5.- Test Local Training

In [1]:
import os
import docker

In [2]:
# Define the path to the directory containing your Dockerfile.
# If you're running on Windows, make sure to use the appropriate path format.
dockerfile_path = "../docker"  # Adjust this path as needed

In [3]:
# Initialize the Docker client (make sure Docker Desktop is running on Windows)
client = docker.from_env()

# Define a tag for the built image.
image_tag = "satellite-object-detection-local:latest"

In [10]:
try:
    # Build the Docker image.
    # 'path' specifies the directory with the Dockerfile.
    # 'tag' sets the name and tag for the resulting image.
    # 'rm=True' removes intermediate containers after a successful build.
    image, build_logs = client.images.build(
        path=dockerfile_path,
        tag=image_tag,
        rm=True
    )

    # Print the build logs to monitor the build process.
    for chunk in build_logs:
        if 'stream' in chunk:
            print(chunk['stream'].strip())

    print("Image built successfully. Tags:", image.tags)

except docker.errors.BuildError as build_err:
    # Catch and print errors that occur during the build process.
    print("Error building the image:", build_err)
except Exception as err:
    # Catch any other exceptions that might occur.
    print("An error occurred:", err)


Step 1/13 : FROM ubuntu:18.04

---> f9a80a55f492
Step 2/13 : ENV DEBIAN_FRONTEND=noninteractive

---> Using cache
---> 7ee1b066a7e3
Step 3/13 : ENV LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH

---> Using cache
---> f450394632fe
Step 4/13 : RUN apt-get update && apt-get install -y     git     wget     unzip     curl     sudo     python3     python3-pip     python3-dev     protobuf-compiler     && rm -rf /var/lib/apt/lists/*

---> Using cache
---> e193457b530b
Step 5/13 : RUN ln -s /usr/bin/python3 /usr/bin/python

---> Using cache
---> 503a38673bae
Step 6/13 : RUN pip3 uninstall -y Cython || true

---> Using cache
---> 4927003512f3
Step 7/13 : RUN git clone --depth 1 https://github.com/tensorflow/models /models

---> Using cache
---> 3f3e29e12b2b
Step 8/13 : WORKDIR /models/research

---> Using cache
---> 539eef410702
Step 9/13 : RUN protoc object_detection/protos/*.proto --python_out=.

---> Using cache
---> 4370f86ca123
Step 10/13 : RUN cp /models/research/object_detec

In [59]:
# Get the current working directory (assuming the notebook is in the 'notebooks' folder)
current_dir = os.getcwd()

# Build the path to the 'src' folder by going one level up and then into 'src'
local_code_path = os.path.abspath(os.path.join(current_dir, '..', 'src'))
local_code_path

'c:\\Users\\walte\\Documents\\Projects\\GitHub\\My_Organizations\\Satellite-Imagery-WSC\\satellite-object-detection\\src'

In [60]:
# Compute local repository paths (assuming the notebook is in the "notebooks" folder)
current_dir = os.getcwd()
local_code_path = os.path.abspath(os.path.join(current_dir, "..", "src"))
local_output_path = os.path.abspath(os.path.join(current_dir, "..", "external", "model_trained"))

In [61]:
# Training parameters
num_train_steps = 1000
sample_1_of_n_eval_examples = 1

In [62]:
# Define container paths
container_code_path = "/workspace"
container_output_path = "/model_output"

In [63]:
# Build the training command using triple quotes
command_to_run = f"""
    python research/object_detection/model_main_tf2.py \
    --pipeline_config_path {os.path.join(container_code_path, "custom_pipeline_file.config")} \
    --model_dir {container_output_path} \
    --alsologtostderr \
    --num_train_steps {num_train_steps} \
    --sample_1_of_n_eval_examples {sample_1_of_n_eval_examples}
"""

In [64]:
# Print paths and command for debugging
print("Local code folder:", local_code_path)
print("Local output folder:", local_output_path)
print("Command to run:", command_to_run)

Local code folder: c:\Users\walte\Documents\Projects\GitHub\My_Organizations\Satellite-Imagery-WSC\satellite-object-detection\src
Local output folder: c:\Users\walte\Documents\Projects\GitHub\My_Organizations\Satellite-Imagery-WSC\satellite-object-detection\external\model_trained
Command to run: 
    python research/object_detection/model_main_tf2.py     --pipeline_config_path /workspace\custom_pipeline_file.config     --model_dir /model_output     --alsologtostderr     --num_train_steps 1000     --sample_1_of_n_eval_examples 1



In [65]:
try:
    # Run a container from your image, mounting local folders into the container
    container = client.containers.run(
        image=image_tag,  # Replace with your image tag
        command=command_to_run,
        volumes={
            local_code_path: {"bind": container_code_path, "mode": "rw"},
            local_output_path: {"bind": container_output_path, "mode": "rw"}
        },
        working_dir=container_code_path,
        detach=True,  # Run the container in detached mode
        remove=True   # Automatically remove the container when it exits
    )

    # Stream and print the logs in real time
    for line in container.logs(stream=True):
        print(line.decode("utf-8").strip())

except Exception as e:
    print("Error:", e)


Traceback (most recent call last):
File "research/object_detection/model_main_tf2.py", line 31, in <module>
from object_detection import model_lib_v2
File "/models/research/object_detection/model_lib_v2.py", line 29, in <module>
from object_detection import eval_util
File "/models/research/object_detection/eval_util.py", line 36, in <module>
from object_detection.metrics import lvis_evaluation
File "/models/research/object_detection/metrics/lvis_evaluation.py", line 23, in <module>
from lvis import results as lvis_results
File "/usr/local/lib/python3.8/site-packages/lvis/__init__.py", line 5, in <module>
from lvis.vis import LVISVis
File "/usr/local/lib/python3.8/site-packages/lvis/vis.py", line 1, in <module>
import cv2
File "/usr/local/lib/python3.8/site-packages/cv2/__init__.py", line 181, in <module>
bootstrap()
File "/usr/local/lib/python3.8/site-packages/cv2/__init__.py", line 153, in bootstrap
native_module = importlib.import_module("cv2")
File "/usr/local/lib/python3.8/importli

In [46]:
# Comando que deseas ejecutar dentro del contenedor
command_to_run = "python research/object_detection/model_main_tf2.py"

In [47]:
try:
    # Ejecuta el contenedor montando el directorio local en /workspace del contenedor
    container = client.containers.run(
        image=image_tag,
        command=command_to_run,
        volumes={
            local_code_path: {  # Directorio local
                'bind': '/workspace',  # Punto de montaje en el contenedor
                'mode': 'rw'
            }
        },
        working_dir="/workspace",  # Define el directorio de trabajo en el contenedor
        detach=True,    # Ejecuta el contenedor en segundo plano
        remove=True     # Elimina el contenedor automáticamente al finalizar
    )

    # Lee e imprime los logs en tiempo real
    for line in container.logs(stream=True):
        print(line.decode('utf-8').strip())

except Exception as e:
    print("Error:", e)


Traceback (most recent call last):
File "research/object_detection/model_main_tf2.py", line 31, in <module>
from object_detection import model_lib_v2
File "/models/research/object_detection/model_lib_v2.py", line 29, in <module>
from object_detection import eval_util
File "/models/research/object_detection/eval_util.py", line 36, in <module>
from object_detection.metrics import lvis_evaluation
File "/models/research/object_detection/metrics/lvis_evaluation.py", line 23, in <module>
from lvis import results as lvis_results
File "/usr/local/lib/python3.8/site-packages/lvis/__init__.py", line 5, in <module>
from lvis.vis import LVISVis
File "/usr/local/lib/python3.8/site-packages/lvis/vis.py", line 1, in <module>
import cv2
File "/usr/local/lib/python3.8/site-packages/cv2/__init__.py", line 181, in <module>
bootstrap()
File "/usr/local/lib/python3.8/site-packages/cv2/__init__.py", line 153, in bootstrap
native_module = importlib.import_module("cv2")
File "/usr/local/lib/python3.8/importli

In [6]:
!wsl --shutdown